In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier

/Users/Nate/anaconda3/envs/py27/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [29]:
#use project clean data - OATH Violations

df = pd.read_csv('processed_data/FINAL_DATA.csv')

In [30]:
df = df[~(df['GEOID'].isnull()) & ~(df['MEDIAN_PERSON_AGE'].isnull())]

In [31]:
df['YEARS_OLD'] = df['YEARS_OLD'].apply(lambda x: 0 if x == 2017. else x)

In [35]:
df.fillna(0, inplace=True)
df.replace([np.inf, -np.inf], 0, inplace=True)

In [36]:
#Determine X and Y and test/train values
Y = df.ix[:,0]
X = df.ix[:, 5:27]


# --------------------
# K-fold CV
# --------------------
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.7, random_state=3)
ts = Y_test.shape[0]

In [61]:
# Decision Tree (CART) Function
maxdepth = 5
#def DT(maxdepth, ts):
# Train
dtc = DecisionTreeClassifier(max_depth = maxdepth)
dtc.fit(X_train, Y_train)

# Predicting
dtc_pred = dtc.predict(X_test)

# Finding mispredicted samples
dtc_verror = np.asarray([int(dtc_pred[i] != Y_test[i]) for i in range(0,ts)])
dtc_error = np.sum(dtc_verror)
dtc_ccidx = np.where(dtc_verror == 0)
dtc_mcidx = np.where(dtc_verror == 1)

perrordtc = float(dtc_error)/ts
return perrordtc


KeyError: 0

In [34]:
# Decision Tree (CART ) with Bagging/Boosting
def bagb(maxdepth, nestimators, ts):
    #Train
    bagb = BaggingClassifier(DecisionTreeClassifier(max_depth=maxdepth), n_estimators=nestimators)
    bagb.fit(X_train,Y_train)

    # Predicting
    bagb_pred = bagb.predict(X_test)

    # Finding mispredicted samples
    bagb_verror = np.asarray([int(bagb_pred[i] != Y_test[i]) for i in range(0,ts)])
    bagb_error = np.sum(bagb_verror)
    bagb_ccidx = np.where(bagb_verror == 0)
    bagb_mcidx = np.where(bagb_verror == 1)

    perrordtb = float(bagb_error)/ts
    return perrordtb


In [35]:
# SVM Function (choose optimal gamma first)

def gammachoice(X_train, Y_train, X_test, Y_test, ts, step):  
    l=[]
    for g in range(1,250, step):
        svm_rbf = svm.SVC(kernel='rbf', gamma=float(g))
        svm_rbf.fit(X_train,Y_train)
        ypred_svm_rbf = svm_rbf.predict(X_test)
        e_svm_rbf = float(np.sum((ypred_svm_rbf[i] != Y_test[i]) for i in range(0,ts)))
        l.append((e_svm_rbf/ts))
    plt.plot(range(0,250, step), l)
    plt.xlabel('gamma')
    plt.ylabel('classification percent error')
    minindex = l.index(min(l))
    #tx = plt.xticks(range(0,250, step))
    print "optimal gamma is:", step*minindex
    return step*minindex

def svmrbf(X_train, Y_train, X_test, Y_test, g, ts): 
    svm_rbf = svm.SVC(kernel='rbf', gamma=g)
    svm_rbf.fit(X_train,Y_train)
    
    ypred_svm_rbf = svm_rbf.predict(X_test)
    
    e_svm_rbf = np.sum((ypred_svm_rbf[i] != Y_test[i]) for i in range(0,ts))
    perrorsvmrbf = float(e_svm_rbf)/ts
    return perrorsvmrbf


In [36]:
# Random Forest (No Boosting Function)

def RFreg(maxdepth, nestimators):
    #Train
    rdf = RandomForestClassifier(max_depth=maxdepth, n_estimators=nestimators, bootstrap = True)
    rdf.fit(X_train,Y_train)

    # Predicting
    rdf_pred = rdf.predict(X_test)

    # Finding mispredicted samples
    rdf_verror = np.asarray([int(rdf_pred[i] != Y_test[i]) for i in range(0,ts)])
    rdf_error = np.sum(rdf_verror)
    rdf_ccidx = np.where(rdf_verror == 0)
    rdf_mcidx = np.where(rdf_verror == 1)

    perrorrf = float(rdf_error)/ts
    return perrorrf

In [37]:
# Random Forest (Boosting Function)

def RFboost(maxdepth, nestimators):
    #Train
    rdfb = AdaBoostClassifier(RandomForestClassifier(max_depth=maxdepth, n_estimators=nestimators, bootstrap = True))
    rdfb.fit(X_train,Y_train)

    # Predicting
    rdf_pred = rdfb.predict(X_test)

    # Finding mispredicted samples
    rdf_verror = np.asarray([int(rdf_pred[i] != Y_test[i]) for i in range(0,ts)])
    rdf_error = np.sum(rdf_verror)
    rdf_ccidx = np.where(rdf_verror == 0)
    rdf_mcidx = np.where(rdf_verror == 1)

    perrorrfb = float(rdf_error)/ts
    return perrorrfb

In [ ]:
# For Loop to determine best coefficients
